In [73]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u 
from math import radians, degrees, sin, cos, asin, acos, sqrt
from opencage.geocoder import OpenCageGeocode
from tqdm import tqdm_notebook as tqdm

plt.style.use("../harsh")

In [14]:
def great_circle(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])    
    return 6371 * (acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2)))

vc_great_circle = np.vectorize(great_circle)

In [3]:
def carbon_simple(travel_distance, travel_class="econ"):
    """
    simple calculator based on https://pics.uvic.ca/sites/default/files/AirTravelWP_FINAL.pdf. 
    takes in travel_distance [km, float] and travel_class [string]
    returns carbon emissions in [kg]
    """
    scaled_distance = 1.08*travel_distance #account for rerouting, holds, etc.
    
    if(scaled_distance < 463):
        return 0.27867*scaled_distance
    
    elif(scaled_distance < 3700):
        if(travel_class == "econ"):
            return 0.16508*scaled_distance
        else: 
            return 0.24761*scaled_distance 
    
    else:
        if(travel_class == "econ"):
            return 0.14678*scaled_distance
        elif(travel_class == "econ_plus"):
            return 0.23484*scaled_distance
        else: 
            return 0.42565*scaled_distance

In [7]:
unis = np.loadtxt("unis", dtype="str", delimiter="/n")

In [8]:
api_key = '46537db450f7457081cb31409f2a9ee5'  # get api key from:  https://opencagedata.com

In [9]:
def return_latlon(uni):
    geocoder = OpenCageGeocode(api_key)
    query = uni
    results = geocoder.geocode(query)
    if(len(results) == 0):
        return None
    else:
        lat = results[0]['geometry']['lat']
        lng = results[0]['geometry']['lng']
        return lat, lng

In [10]:
all_uni_latlons = np.zeros((len(unis), 2))
for i in tqdm(range(len(unis))):
    try:
        all_uni_latlons[i,0], all_uni_latlons[i,1] = return_latlon(unis[i])
    except:
        print("%s DOESN'T EXIST" % unis[i])
        all_uni_latlons[i,0], all_uni_latlons[i,1] = np.nan, np.nan

In [51]:
honolulu_lat = 21.3069
honolulu_lon = 157.8583

In [74]:
def calculate_carbon_footprint(number_of_people, conference_location):
    choose_attendee_locs = all_latlons[np.random.choice(len(all_uni_latlons), number_of_people)]
    all_distances = vc_great_circle(choose_attendee_locs[:,1], choose_attendee_locs[:,0], honolulu_lon, honolulu_lat)*2 

    s = 0
    for i in tqdm(range(len(all_distances))):
        s += carbon_simple(all_distances[i])
    
    return s